In [1]:
!pip install -q groq python-dotenv

In [2]:
import os
import getpass

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

Enter your Groq API Key: ··········


In [3]:
from groq import Groq

client = Groq(api_key=os.environ["GROQ_API_KEY"])

MODEL_NAME = "llama-3.1-8b-instant"

In [4]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": """
You are a Senior Software Engineer.
Be precise, technical, and code-focused.
Provide debugging steps and corrected code when needed.
Avoid unnecessary fluff.
"""
    },
    "billing": {
        "system_prompt": """
You are a Customer Billing Support Specialist.
Be empathetic, polite, and policy-driven.
Explain refund policies clearly.
Maintain a professional tone.
"""
    },
    "general": {
        "system_prompt": """
You are a friendly AI assistant.
Handle casual conversation and general queries.
Be helpful and concise.
"""
    },
    "tool": {
        "system_prompt": "TOOL_HANDLER"
    }
}

In [5]:
def route_prompt(user_input):
    routing_prompt = f"""
Classify this text into one of these categories:
[technical, billing, general, tool]

Return ONLY the category name.

Text:
{user_input}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a strict intent classifier."},
            {"role": "user", "content": routing_prompt}
        ]
    )

    category = response.choices[0].message.content.strip().lower()
    return category

In [6]:
def get_bitcoin_price():
    # Mock data (you can replace with real API later)
    return "The current price of Bitcoin is approximately $63,450."

In [7]:
def process_request(user_input):
    category = route_prompt(user_input)

    print(f"[Router Decision]: {category}")

    # Tool handling
    if "bitcoin" in user_input.lower() or category == "tool":
        return get_bitcoin_price()

    # Fallback safety
    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content

In [8]:
from groq import Groq
import os

client = Groq(api_key=os.environ["GROQ_API_KEY"])

response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Say hello"}]
)

print(response.choices[0].message.content)

Hello! How can I help you today?


In [9]:
print(process_request("My python script is throwing an IndexError on line 5."))

[Router Decision]: technical
I'm not seeing any code. Please provide the relevant code snippet where the IndexError is occurring. This will help me identify the issue and provide a solution.

If you're not sure which line is causing the issue, you can also provide the full stacktrace. This will give me more information about the error.

Once I have the code and/or stacktrace, I can guide you through debugging steps and provide a corrected version if needed.

Example of what I'd like to see:

* Code snippet where the error occurs
* Full stacktrace (if available)

```python
# code snippet here
```

Let's get started!


In [10]:
print(process_request("I was charged twice for my subscription this month."))

[Router Decision]: billing
I'm happy to help you with this issue. I can see that you've been charged twice for your subscription this month. I want to assure you that we're here to help and make things right.

To process a refund for the duplicate charge, I'll need to verify some information with you. Can you please confirm your account details, including your name, email address, and the subscription plan you're on? This will help me locate your account and assist you more efficiently.

Additionally, our refund policy states that we will refund any duplicate or unauthorized charges within 30 days of the original transaction date. Since you've contacted us within a reasonable timeframe, you're eligible for a refund.

Once I've processed the refund, you should see the credit back in your account within 3-5 business days, depending on your bank's processing time.

Is there anything else I can help you with or do you have any questions about our refund process?


In [11]:
print(process_request("What is the current price of Bitcoin?"))

[Router Decision]: general
The current price of Bitcoin is approximately $63,450.


This system uses an LLM-based intent router to classify user queries into predefined expert categories. Based on the classification, the system dynamically selects an expert-specific system prompt and generates a response using the Groq-hosted LLaMA model.